In [1]:
import glob
import os
from datetime import datetime, timedelta, date
from os import path

from igraph import *

import seaborn as sns
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [24]:
# Various datasets to visualize

option = 1

if option == 1:
    base_folder = "./osc2020"
#     ext = "csv" # Extension of log file, either log or txt
#     ntot = 20   # Total number of participants
#     vstep = 2   # Time step for visualization
#     ostep = 1   # Time step for output to R
#     infer_new_cases_from_inf = True # Time for "new case" is inferred either from infection or removal time
#     asympt = 2  # Asymptomatic period
#     sympt = 2   # Symptomatic period
#     start_sim = datetime.strptime("2020-03-05 13:30", '%Y-%m-%d %H:%M')
#     stop_sim = datetime.strptime("2020-03-05 14:15", '%Y-%m-%d %H:%M')

In [25]:
# Some config

# Coded outcomes:
# 0 = no infection
# 1 = index case
# 2 = transmission
color_dict = {0: "skyblue", 1: "lime", 2: "crimson"}

data_folder = path.join(base_folder, "data")
output_folder = path.join(base_folder, "output")
if not path.exists(output_folder):
    os.mkdir(output_folder)
    
# viz_delta = timedelta(minutes=vstep)
# out_delta = timedelta(minutes=ostep)  

In [88]:
# Load participants and histories

users = pd.read_csv(path.join(data_folder, "participants.csv")) 
histories = pd.read_csv(path.join(data_folder, "histories.csv"))

p2pToSim = pd.Series(users.sim_id.values, index=users.p2p_id).to_dict()

# dfc = participants[participants["sim_id"] == 5]
# dictc = pd.Series(dfc.id.values,index=dfc.p2p_id).to_dict()

# dfs = participants[participants["sim_id"] == 6]
# dicts = pd.Series(dfs.id.values,index=dfs.p2p_id).to_dict()

# center group
gr = 5 
fn = "contacts-center.pdf"

#south group
# gr = 6 
# fn = "contacts-south.pdf"

# north group
# gr = 7 
# fn = "contacts-north.pdf"

dfn = users[users["sim_id"] == gr]
hn = histories[histories["sim_id"] == gr]
cn = hn[hn["type"] == "contact"]

dfn
p2pToId = pd.Series(dfn.id.values, index=dfn.p2p_id).to_dict()
idTop2p = pd.Series(dfn.p2p_id.values, index=dfn.id).to_dict()
        
user_index = {}
idx = 0
for kid in idTop2p:
    user_index[kid] = idx
    idx += 1

print(len(dfn))
print(len(idTop2p))    
print(len(p2pToId))
print(len(user_index))

47
47
47
47


In [89]:
# Generate contacts

node0 = cn.user_id.values
node1 = cn.peer_id.values
time = cn.time.values
length = cn.contact_length.values

contacts = {}
for id0, p2p1, t, l01 in zip(node0, node1, time, length):
    n0 = user_index[id0]
    if not p2p1 in p2pToId:
        print("Possible contact with group", p2pToSim[p2p1])
        continue
    n1 = user_index[p2pToId[p2p1]]
    
    t = datetime.fromtimestamp(t)
    
    if n1 < n0:
        t = n1
        n1 = n0
        n0 = t
    p01 = (n0, n1)
    if p01 in contacts:
        c = contacts[p01]
    else: 
        c = 0
    contacts[p01] = c + round(l01 / (60 * 1000))
    
print(contacts)

{}


In [90]:
# Generate infections and outcomes

inf = hn[hn["type"] == "infection"]
infMap = pd.Series(inf.inf.values, index=inf.user_id).to_dict()
outcomes = [0] * len(dfn)

for kid in infMap:
    src = infMap[kid]
    idx = user_index[kid]
    if "CASE0" in src:
        outcomes[idx] = 1
    if "PEER" in src:
        outcomes[idx] = 2

In [91]:
nvert = len(user_index)

edges = []
weights = []
edgesw = []
for p in contacts:
    n0 = p[0]
    n1 = p[1]
    w = contacts[p]
    if 0 < w:
        edges += [(n0, n1)]
        weights += [w]
        edgesw += [(n0, n1, w)]

# https://stackoverflow.com/a/50430444
g = Graph(directed=False)
g.add_vertices(nvert)
g.add_edges(edges)
g.es['weight'] = weights
# g.es['label'] = weights

# g = Graph.TupleList(edgesw, weights=True)

g.vs["outcome"] = outcomes
g.vs["color"] = [color_dict[out] for out in g.vs["outcome"]]

print(g.is_weighted())

style = {}
style["bbox"] = (800, 800)
style["margin"] = 15
style["vertex_size"] = 8
style["vertex_label_size"] = 8
style["edge_curved"] = False
# https://igraph.org/c/doc/igraph-Layout.html#igraph_layout_graphopt
style["layout"] = g.layout_fruchterman_reingold(weights=g.es["weight"])

p = plot(g, os.path.join(output_folder, fn), **style)

True


In [92]:
print("Number of vertices in the graph:", g.vcount())
print("Number of edges in the graph", g.ecount())
print("Is the graph directed:", g.is_directed())
print("Maximum degree in the graph:", g.maxdegree())
# print("Adjacency matrix:\n", g.get_adjacency())

Number of vertices in the graph: 47
Number of edges in the graph 0
Is the graph directed: False
Maximum degree in the graph: 0
